In [1]:
####################################
## A. Librerias
####################################
import os
import time
import json

import datetime
import pandas as pd

from utils.utils import *
from utils.paths import CHROMEDRIVER_PATH, PROJECT_PATH

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

####################################
## B. Funciones
####################################

def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def start_har(proxy, name):
    options = {
        'captureHeaders': True, 
        'captureContent': True, 
        'captureBinaryContent': True, 
        'captureCookies': True, 
        'captureHeadersSize': -1, 
        'captureMaxSize': -1, 
        'captureBinaryContentMaxLength': -1 # Tamaño máximo del contenido binario
    }
    proxy.new_har(name, options=options)

ESTAMOS EN --->  darwin
PROJECT_PATH:  /Users/nahumfg/Projects/GitHubProjects/RenacytScraper
CHROME_DRIVER_PATH:  /Users/nahumfg/Projects/GitHubProjects/RenacytScraper/chromedriver/darwin/chromedriver



In [2]:
########################
## C. Ejecución
########################

# 1. Seleccionar modo
HEADLESS = True
PRINT_VIEW = False

# 2. Eliminar todos los archivos de la carpeta
try:
    files = get_files(os.path.join(os.getcwd(),'outputs','jsons','renacyt_profile'), '.')
    for file in files:
        os.remove(file)
    print('Se eliminaron los archivos de la carpeta')
except:
    pass

Se eliminaron los archivos de la carpeta


In [3]:
######################
## D. Leer los jsons
######################

# 1. Leer los jsons y guardarlos en un dataframe
df_renacyt= pd.read_parquet(os.path.join(os.getcwd(),'outputs','parquets','renacyt.parquet'))
df_renacyt = df_renacyt[df_renacyt['ctiVitae'].notnull()]
df_renacyt = df_renacyt[df_renacyt['ctiVitae'] != 'nan']
df_renacyt = df_renacyt[df_renacyt['ctiVitae'] != 'None']
df_renacyt = df_renacyt[df_renacyt['ctiVitae'] != '']
df_renacyt['ctiVitae'] = df_renacyt['ctiVitae'].astype(str)
df_renacyt.drop_duplicates(subset=['ctiVitae'], inplace=True)
df_renacyt = df_renacyt.astype(str)

# 2. Seleccionar la columna ctiVitae y crear otra columna con el url + ctiVitae
df_renacyt = df_renacyt[['orcid','ctiVitae']]
df_renacyt['url'] = 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=' + df_renacyt['ctiVitae']
df_renacyt = df_renacyt[['orcid','url']]

df_renacyt = df_renacyt.head(10)
df_renacyt

,orcid,url
738,0000-0003-1583-7113,https://servicio-renacyt.concytec.gob.pe/ficha...
5797,None,https://servicio-renacyt.concytec.gob.pe/ficha...
5749,0000-0001-9740-5490,https://servicio-renacyt.concytec.gob.pe/ficha...
3308,0000-0002-8301-5536,https://servicio-renacyt.concytec.gob.pe/ficha...
6842,0000-0002-8207-9714,https://servicio-renacyt.concytec.gob.pe/ficha...
4765,0000-0002-9695-5746,https://servicio-renacyt.concytec.gob.pe/ficha...
5968,0000-0003-3338-2503,https://servicio-renacyt.concytec.gob.pe/ficha...
5004,0000-0003-2259-5327,https://servicio-renacyt.concytec.gob.pe/ficha...
7204,0000-0002-8069-2101,https://servicio-renacyt.concytec.gob.pe/ficha...
5917,0000-0001-5560-9241,https://servicio-renacyt.concytec.gob.pe/ficha...


In [4]:
######################
## E. Ejecutar
######################

# 1. Iniciar el driver, el servidor proxy y el proxy
driver, server, proxy = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=PRINT_VIEW, headless=HEADLESS)

# 2. Iterar sobre las urls
i = 0
for orcid, url in df_renacyt.values.tolist(): 
    try:
        # A. Iniciar har
        start_har(proxy, 'renacyt_profile')
        
        # B. Ingresar a la url
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "elementor-widget-container")))
        print(f'{i+1}. {url}')

        # C. Obtener el har
        time.sleep(3)
        har = proxy.har

        # D. Guardar el har
        for entrie in har.get('log').get('entries'):
            if '"formacionesAcademicas"' in str(entrie):
                string_content = entrie.get('response').get('content').get('text')
                data_list = json.loads(string_content)
                with open(os.path.join(os.getcwd(),'outputs','jsons'
                                       ,'renacyt_profile','renacyt_profile_' 
                                       + str(i).zfill(2) 
                                       + '_'+ orcid +'_.json'), 'w') as json_file:
                    json.dump(data_list, json_file, indent=4)
                
        # E. Actualizar el contador
        i += 1
    except:
        pass

Se inicia el servidor en Mac o Linux
Usando el chromedriver_path: /Users/nahumfg/Projects/GitHubProjects/RenacytScraper/chromedriver/darwin/chromedriver
1. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=10
2. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=13
3. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=16
4. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=17
5. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=19
6. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=21
7. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=25
8. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=26
9. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=34
10. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=43


In [5]:
stop_chrome_driver(driver, server, proxy)

# Eliminar los archivos bmp.log y server.log
try:
    os.remove('bmp.log')
    os.remove('server.log')
    print("Se eliminaron los archivos bmp.log y server.log")
except:
    pass

Se eliminaron los archivos bmp.log y server.log
